In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
import os
import pandas as pd
import numpy as np
import cv2

In [2]:
base_dir = r'E:\AB\UEM\year 3\SEM 6\pneumonia_binary\train'
img_h,img_w= (164,164)
batch_size=32
epochs=3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen= ImageDataGenerator(rescale=1./255,validation_split=0.2)

In [3]:
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
#from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import PReLU
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras.callbacks import ReduceLROnPlateau

import keras
from tensorflow.python.keras.metrics import Metric
base_model= ResNet50(include_top=False, weights='imagenet',
                                       input_tensor=None, input_shape=(img_h,img_w,3), pooling='avg')
 
for layer in base_model.layers[:-7]:
    layer.trainable=False

 
model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.4))


model.add(Dense(2,activation='sigmoid'))
#model.add(Dense(1))
#model.add(Activation("sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23,591,810
Trainable params: 1,059,842
Non-trainable params: 22,531,968
_________________________________________________________________


In [4]:
for layer in base_model.layers:
    print(layer,"-->",layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x0000013E2901BCD0> --> False
<keras.layers.convolutional.ZeroPadding2D object at 0x0000013E29072640> --> False
<keras.layers.convolutional.Conv2D object at 0x0000013E290729D0> --> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000013E290E37C0> --> False
<keras.layers.core.activation.Activation object at 0x0000013E290E34C0> --> False
<keras.layers.convolutional.ZeroPadding2D object at 0x0000013E29121D60> --> False
<keras.layers.pooling.MaxPooling2D object at 0x0000013E29176CA0> --> False
<keras.layers.convolutional.Conv2D object at 0x0000013E29198DC0> --> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000013E29193C70> --> False
<keras.layers.core.activation.Activation object at 0x0000013E290E3160> --> False
<keras.layers.convolutional.Conv2D object at 0x0000013E291A1400> --> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000

In [5]:
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau


reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=3,
                                         cooldown=2,
                                         min_lr=1e-10,
                                         verbose=1)

checkpoint =tf.keras.callbacks.ModelCheckpoint(filepath="resfinal.h5", 
                            monitor='val_accuracy',
                            verbose=1,
                            save_best_only=True, 
                            save_weights_only=False, 
                            mode='auto',
                            save_freq='epoch')


import time
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

time_callback = TimeHistory()

callbacks = [reduce_learning_rate, checkpoint, time_callback]
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile( loss='binary_crossentropy',optimizer= optimizer, metrics=['accuracy',keras.metrics.Precision(),keras.metrics.Recall()])

F:\Anaconda\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [6]:
val_dir = r'E:\AB\UEM\year 3\SEM 6\pneumonia_binary\val'
train_generator = datagen.flow_from_directory(
        base_dir,  # This is the source directory for training images
        target_size=(img_h,img_w),  
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',
        #color_mode="rgb",
        shuffle=True)


validation_generator = datagen.flow_from_directory(
         val_dir,     
        target_size=(img_h, img_w),
        batch_size=batch_size,
        class_mode='categorical',
        #color_mode="rgb",
        subset='validation',
        shuffle=False)

Found 3354 images belonging to 2 classes.
Found 207 images belonging to 2 classes.


In [7]:
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples//batch_size, 
      epochs=1,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples//batch_size,  
      callbacks=callbacks,
      verbose=1)

104/104 [==============================] - ETA: 0s - loss: 0.4499 - accuracy: 0.7974 - precision: 0.7920 - recall: 0.7911
Epoch 00001: val_accuracy improved from -inf to 0.86979, saving model to resfinal.h5


F:\Anaconda\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


104/104 [==============================] - 204s 2s/step - loss: 0.4499 - accuracy: 0.7974 - precision: 0.7920 - recall: 0.7911 - val_loss: 0.4885 - val_accuracy: 0.8698 - val_precision: 0.8852 - val_recall: 0.8438 - lr: 0.0010


In [ ]:
model.evaluate(validation_generator,verbose=1)

In [19]:
history_dict = history.history
print(history_dict.keys())

dict_keys(['loss', 'lr'])


In [15]:
accu= history.history['val_accuracy']
print(max(accu),"at epoch number-",accu.index(max(accu))+1)


#print(f" — val_f1: {_val_f1} — val_precision: {_val_precision} — val_recall _val_recall")

0.9010416865348816 at epoch number- 1


In [34]:
img = cv2.imread("E:\\AB\\UEM\\year 3\\SEM 6\\pneumonia_binary\\test\\opacity\\person16_virus_47.jpeg") 
image_resized= cv2.resize(img, (img_h,img_w))
'''cv2.imshow('CT scan', img)
cv2.waitKey(0)'''
img=np.expand_dims(image_resized,axis=0)
np.set_printoptions(suppress=True)
pred=model.predict(img)
#print(pred);
yn=np.argmax(pred,axis=1)
if yn==0:
    print("Pneumonia")
else :
    print("Not")

Pneumonia


In [11]:
yn=np.argmax(pred,axis=1)
classes = []
for subdir in sorted(os.listdir(base_dir)):
    if os.path.isdir(os.path.join(base_dir, subdir)):
        classes.append(subdir)
if yn==0:
    print("pneumonia")
else :
    print("Not pneumonia")

Not pneumonia


In [15]:
yn=np.argmax(pred,axis=1)
classes = []
for subdir in sorted(os.listdir(base_dir)):
    if os.path.isdir(os.path.join(base_dir, subdir)):
        classes.append(subdir)
yn

array([0], dtype=int64)

In [48]:
import keras
from keras.utils.np_utils import to_categorical
y_proba = model.predict(img)
y_classes = y_proba.argmax(axis=-1)
y_classes

array([0], dtype=int64)

In [47]:
timings= time_callback.times
n=sum(timings)
print("Total time taken is ",sum(timings)//60," minutes and ",sum(timings)%60," seconds")

Total time taken is  6.0  minutes and  15.173778057098389  seconds


In [32]:
model2= tf.keras.models.load_model("resfinal.h5")
y_pred= model2.predict(validation_generator,verbose=1)
y_true= validation_generator.classes
y_pred2= np.argmax(y_pred,axis=1)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_true,y_pred2))
from sklearn.metrics import confusion_matrix

cf=confusion_matrix(yn, pred)

df_cm = pd.DataFrame(cf, range(2), range(2))
df_cm.to_csv('ResNet50 prj'+'.csv',index=False)

1/1 [==============================] - 24s 24s/step
0.2560386473429952


ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets

In [18]:
cf

array([[ 53,   0],
       [154,   0]], dtype=int64)

In [25]:
ac = max(accu)
ac=ac*100
ac

89.86442685127258

In [31]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(yn, pred, average='macro')

ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets